# Notebook for plotting the main shapes along the data workflow
### Jupyter notebook need to be improved
The considered shapes to be plotted refer to the output of the following scripts:
- build_shapes
- download_osm_data
- clean_osm_data
- build_osm_network
- base_network
- build_bus_regions

## Load packages

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from shapely.geometry import LineString, Point
from shapely.wkt import loads
from collections import Counter

import os
import sys

sys.path.append("../")  # to import helpers
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

## Build shapes

Load build_shapes output and plot them

In [ ]:
# load shapes
continent = gpd.read_file("resources/country_shapes.geojson")
off_shore = gpd.read_file("resources/offshore_shapes.geojson")
gadm_shapes = gpd.read_file("resources/gadm_shapes.geojson")

In [ ]:
ax = gadm_shapes.boundary.plot(color="blue")
# off_shore.boundary.plot(ax=ax, color="green")
# continent.boundary.plot(ax=ax, color="red")
plt.axis("off")
plt.savefig("shapes_ZA.png")

## download_osm_data

Load and plot the raw data downloaded by the power extractor package

In [ ]:
# plot output download_osm_data
df_substations_osm_download = gpd.read_file(
    os.getcwd() + "/data/raw/africa_all_raw_substations.geojson"
)
df_lines_osm_download = gpd.read_file(
    os.getcwd() + "/data/raw/africa_all_raw_lines.geojson"
)

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
df_substations_osm_download.plot(ax=ax, color="orange")
df_lines_osm_download.plot(ax=ax, color="yellow")
# df_lines_osm_download[df_lines_osm_download.Country == "TD"].plot(ax=ax, color="yellow")
# plt.axis("off")
# plt.savefig("network.png")

In [ ]:
# df_lines_osm_download[df_lines_osm_download.Country == "TD"]#.geometry.boundary#.dropna(subset=["tags.voltage"])

In [ ]:
# analyses buses

cnt_buses = Counter(df_substations_osm_download["tags.voltage"])

cnt_buses_outliers = {
    key: value
    for (key, value) in cnt_buses.items()
    if key is None or not key.isnumeric()
}

cnt_buses_outliers

# sum(list(cnt_buses.values()))

In [ ]:
# analyses lines

cnt_lines = Counter(df_lines_osm_download["tags.voltage"])

cnt_lines_outliers = {
    key: value
    for (key, value) in cnt_lines.items()
    if key is None or not key.isnumeric()
}

cnt_lines_outliers

## clean_osm_data

Load and plot the data filtered by the clean_osm_data script

In [ ]:
# plot output clean_osm_data
df_substations_osm_clean = gpd.read_file(
    os.getcwd() + "/data/clean/africa_all_substations.geojson"
)
df_lines_osm_clean = gpd.read_file(os.getcwd() + "/data/clean/africa_all_lines.geojson")

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
df_substations_osm_clean.plot(ax=ax, color="orange")
df_lines_osm_clean.plot(ax=ax, color="yellow")

## build_osm_network

Load and plot the data of the network constructed by the build_osm_network script

In [ ]:
# plot output build_osm_network
df_substations_osm_build_network_pd = pd.read_csv(
    os.getcwd() + "/data/base_network/africa_all_buses_build_network.csv"
)
df_lines_osm_build_network_pd = pd.read_csv(
    os.getcwd() + "/data/base_network/africa_all_lines_build_network.csv"
)

df_substations_osm_build_network_pd["geometry"] = gpd.points_from_xy(
    df_substations_osm_build_network_pd.lon, df_substations_osm_build_network_pd.lat
)
# df_lines_osm_build_network_pd.dropna(subset="geometry", inplace=True)
df_lines_osm_build_network_pd["geometry"] = df_lines_osm_build_network_pd[
    "geometry"
].map(loads)

# df_substations_osm_build_network_pd = df_substations_osm_build_network_pd[df_substations_osm_build_network_pd.country=="EH"]
# df_lines_osm_build_network_pd = df_lines_osm_build_network_pd[df_lines_osm_build_network_pd.country=="EH"]

df_substations_osm_build_network = gpd.GeoDataFrame(
    geometry=df_substations_osm_build_network_pd.geometry, crs="epsg:4326"
)
df_lines_osm_build_network = gpd.GeoDataFrame(
    geometry=df_lines_osm_build_network_pd.geometry, crs="epsg:4326"
)

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
df_substations_osm_build_network.plot(ax=ax, color="orange")
df_lines_osm_build_network.plot(ax=ax, color="yellow")

In [ ]:
df_substations_osm_build_network_pd[
    np.abs(df_substations_osm_build_network_pd["lat"] - 7.4710) < 0.001
]

In [ ]:
df_lines_osm_build_network_pd[
    (np.abs(df_lines_osm_build_network_pd.bus0_lat - 7.471) < 0.01)
    | (np.abs(df_lines_osm_build_network_pd.bus1_lon - 7.471))
]

In [ ]:
df_lines_osm_build_network_pd[
    (df_lines_osm_build_network_pd.bus0 == 97)
    | (df_lines_osm_build_network_pd.bus1 == 97)
]

In [ ]:
import hvplot.pandas

df_substations_osm_build_network.hvplot(
    geo=True,
    size=10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles="CartoLight",
    hover_cols=["bus_id"],
    color="orange",
) * df_lines_osm_build_network.hvplot(geo=True, alpha=0.4, hover_cols=["line_id"]).opts(
    active_tools=["pan", "wheel_zoom"]
)

In [ ]:
df_substations_osm_build_network_pd

In [ ]:
ax = continent.boundary.plot(figsize=(10, 10))
# off_shore.plot(ax=ax)
df_substations_osm_build_network.plot(ax=ax, color="blue")
df_lines_osm_build_network.plot(ax=ax, color="orange")
plt.axis("off")
plt.savefig("map.png")

In [ ]:
df_lines_osm_build_network_pd[
    df_lines_osm_build_network_pd.country == "NG"
]  # .geometry.boundary#.dropna(subset=["tags.voltage"])
# df_lines_osm_build_network_pd[df_lines_osm_build_network_pd.line_id == "452716570-1"].copy().iloc[0]
# .geometry[0].bounds
# .boundary#.iloc[7402]

## base_network

Load and plot the data of the network model implemented in PyPSA by the base_network script

In [ ]:
# plot output base_network
n = pypsa.Network(os.getcwd() + "/networks/base.nc")

lines_raw = n.lines.copy()
lines_raw["geometry"] = lines_raw["geometry"].map(loads)
# [LineString([Point(x.bus0_lon, x.bus0_lat), Point(x.bus1_lon, x.bus1_lat)]) for index, x in lines_raw.iterrows()
#     if (x.bus0_lat is not None) and (x.bus0_lon is not None) and (x.bus1_lat is not None) and (x.bus1_lon is not None)]
lines = gpd.GeoDataFrame(geometry=lines_raw.geometry, crs="epsg:4326")

buses_raw = n.buses.copy()
buses_raw["geometry"] = gpd.points_from_xy(buses_raw.lon, buses_raw.lat)
buses = gpd.GeoDataFrame(geometry=buses_raw.geometry, crs="epsg:4326")

continent = gpd.read_file("resources/country_shapes.geojson")
off_shore = gpd.read_file("resources/offshore_shapes.geojson")

ax = continent.plot(figsize=(10, 10))
off_shore.plot(ax=ax)
buses.plot(ax=ax, color="orange")
lines.plot(ax=ax, color="yellow")

In [ ]:
lines_raw[
    lines_raw.country == "NG"
]  # .geometry.boundary#.dropna(subset=["tags.voltage"])

## build_bus_regions

Plot the shapes obtained with the build_bus_regions script

In [ ]:
# plot output build_bus_regions

bus_regions_continent = gpd.read_file("resources/regions_onshore.geojson")
bus_regions_off_shore = gpd.read_file("resources/regions_offshore.geojson")

ax = bus_regions_continent.plot(figsize=(10, 10))
# bus_regions_continent[bus_regions_continent.country.map(lambda x: x in ["TD", "SS", "CF", "SO"])].plot(ax=ax, color="green")
bus_regions_off_shore.plot(ax=ax, color="red")
plt.axis("off")
plt.savefig("bus_regions.png")

In [ ]:
bus_regions_continent[bus_regions_continent.country == "CF"].plot()